In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Hardcode the job position and location
ville = 'international'

# Initialize the list to store job data
data = []

# Hardcode the URL with specific values for job position and location
url = f'https://www.bayt.com/en/international/jobs/'

# Function to extract job data from a job listing
def extract_job_data(job_listing):
    # Extract job ID from the 'data-job-id' attribute
    job_id = job_listing['data-job-id']
    job_details_url = f'https://www.bayt.com/en/{ville}/jobs/?jobId={job_id}'
    detailsResponse = requests.get(job_details_url)
    # Parse the HTML content of the response using BeautifulSoup
    soupDetails = BeautifulSoup(detailsResponse.content, 'html.parser')
    # Extract the new job description from the specific element
    job_description_element = soupDetails.find('div', class_='jb-descr m10t t-small').text.strip()
    # Extract other relevant information
    job_title = job_listing.find('h2', class_='jb-title').text.strip()
    company = job_listing.find('b', class_='jb-company').text.strip()
    location = job_listing.find('span', class_='jb-loc').text.strip()
    job_description = job_description_element
    date_posted = job_listing.find('div', class_='jb-date').text.strip()

    # Store the extracted data in a dictionary
    job_data = {
        'Job ID': job_id,
        'Job Title': job_title,
        'Company': company,
        'Location': location,
        'Job Description': job_description,
        'Date Posted': date_posted,
        'Fraudulent': 0
    }

    return job_data

# Loop to load multiple pages
page = 1
while True:
    # Send a GET request to the URL and store the response
    response = requests.get(url, params={'page': page})

    # Parse the HTML content of the response using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Loop through each job listing on the page
    for job_listing in soup.find_all('li', {'data-js-job': True}):
        # Extract job data and append to the 'data' list
        data.append(extract_job_data(job_listing))

    # Check if there are more results by looking for the "More Results" button
    more_results_button = soup.find('a', class_='jsAjaxLoad btn u-expanded')
    if more_results_button:
        # Increment the page number for the next request
        page += 1
    else:
        # Break the loop if no more results are found
        break

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(data)

# Print or save the DataFrame as needed
print(df)

# Save the DataFrame to a CSV file
df.to_csv('jobs_posts.csv', index=False, encoding='utf-8')

